# Analyzing the Graph of Thrones with Neo4j

In [1]:
#! pip install py2neo
from py2neo import Graph, authenticate
authenticate("localhost:7474", "neo4j", "Paparasta1+")
graph = Graph()

## Import into Neo4j


In [2]:
graph.run("CREATE CONSTRAINT ON (c:Character) ASSERT c.name IS UNIQUE;")

for record in graph.run('''
LOAD CSV WITH HEADERS FROM "https://www.macalester.edu/~abeverid/data/stormofswords.csv" AS row
MERGE (src:Character {name: row.Source})
MERGE (tgt:Character {name: row.Target})
MERGE (src)-[r:INTERACTS]->(tgt)
SET r.weight = toInt(row.Weight)
RETURN count(*) AS paths_written
'''):
    print(record)

('paths_written': 352)


In [3]:
for r in graph.run('''
MATCH p=(:Character)-[:INTERACTS]-(:Character)
RETURN p
'''):
    print(r)

('p': (cbc8a08)-[:INTERACTS {weight:31}]->(bc003b2))
('p': (cbc8a08)-[:INTERACTS {weight:5}]->(ecd017f))
('p': (cbc8a08)<-[:INTERACTS {weight:4}]-(f2d24e3))
('p': (cbc8a08)<-[:INTERACTS {weight:4}]-(cb2f184))
('p': (cbc8a08)<-[:INTERACTS {weight:30}]-(e1376bc))
('p': (a22ae5a)-[:INTERACTS {weight:8}]->(f534d4d))
('p': (a22ae5a)-[:INTERACTS {weight:5}]->(e73340b))
('p': (a22ae5a)-[:INTERACTS {weight:6}]->(cb2f184))
('p': (a22ae5a)-[:INTERACTS {weight:18}]->(eb5c770))
('p': (d0cec03)-[:INTERACTS {weight:5}]->(d24915c))
('p': (d0cec03)<-[:INTERACTS {weight:15}]-(e1376bc))
('p': (d0cec03)<-[:INTERACTS {weight:9}]-(b2b047b))
('p': (b84d643)-[:INTERACTS {weight:5}]->(a7d79f3))
('p': (bc61210)-[:INTERACTS {weight:11}]->(eb5c770))
('p': (bc61210)-[:INTERACTS {weight:7}]->(df35669))
('p': (bc61210)-[:INTERACTS {weight:43}]->(a5b0942))
('p': (bc61210)-[:INTERACTS {weight:5}]->(c8b0db5))
('p': (bc61210)-[:INTERACTS {weight:6}]->(c37654f))
('p': (bc61210)-[:INTERACTS {weight:9}]->(c43f8c8))
('p': 

![](http://www.lyonwj.com/public/img/got-graph-full.png)

## Analyzing the network

### Number of characters

In [4]:
for record in graph.run("MATCH (c:Character) RETURN count(c) AS num"):
    print(record)

('num': 107)


### Summary statistics

In [5]:
for record in graph.run('''
MATCH (c:Character)-[:INTERACTS]->()
WITH c, count(*) AS num
RETURN min(num) AS min, max(num) AS max, avg(num) AS avg_characters, stdev(num) AS stdev
'''):
    print(record)

('min': 1, 'max': 24, 'avg_characters': 4.957746478873239, 'stdev': 6.227672391875085)


### Diameter of the network

In [6]:
for r in graph.run('''
// Find maximum diameter of network
// maximum shortest path between two nodes
MATCH (a:Character), (b:Character) WHERE id(a) > id(b)
MATCH p=shortestPath((a)-[:INTERACTS*]-(b))
RETURN length(p) AS len, extract(x IN nodes(p) | x.name) AS path
ORDER BY len DESC LIMIT 4
'''):
    print(r)

('len': 6, 'path': ['Illyrio', 'Belwas', 'Barristan', 'Jaime', 'Gregor', 'Oberyn', 'Amory'])
('len': 6, 'path': ['Illyrio', 'Belwas', 'Barristan', 'Jaime', 'Arya', 'Bran', 'Jojen'])
('len': 6, 'path': ['Illyrio', 'Belwas', 'Barristan', 'Robert', 'Stannis', 'Davos', 'Shireen'])
('len': 6, 'path': ['Illyrio', 'Belwas', 'Barristan', 'Jaime', 'Arya', 'Bran', 'Luwin'])


### Shortest path

In [9]:
for r in graph.run('''
// Shortest path from Catelyn Stark to Khal Drogo
MATCH (catelyn:Character {name: "Catelyn"}), (drogo:Character {name: "Drogo"})
MATCH p=shortestPath((catelyn)-[INTERACTS*]-(drogo))
RETURN p
'''):
    print(r)

('p': (a9c6b0b)-[:INTERACTS {weight:4}]->(c8b0db5)-[:INTERACTS {weight:16}]->(cb2f184)<-[:INTERACTS {weight:5}]-(b500c3e)-[:INTERACTS {weight:18}]->(c0ede8b))


![](http://www.lyonwj.com/public/img/shortestpath-got.png)

### All shortest paths

In [10]:
for r in graph.run('''
// All shortest paths from Catelyn Stark to Khal Drogo
MATCH (catelyn:Character {name: "Catelyn"}), (drogo:Character {name: "Drogo"})
MATCH p=allShortestPaths((catelyn)-[INTERACTS*]-(drogo))
RETURN p
'''):
    print(r)

('p': (a9c6b0b)-[:INTERACTS {weight:4}]->(c8b0db5)-[:INTERACTS {weight:16}]->(cb2f184)<-[:INTERACTS {weight:5}]-(b500c3e)-[:INTERACTS {weight:18}]->(c0ede8b))
('p': (a9c6b0b)-[:INTERACTS {weight:4}]->(f2d24e3)<-[:INTERACTS {weight:5}]-(cb2f184)<-[:INTERACTS {weight:5}]-(b500c3e)-[:INTERACTS {weight:18}]->(c0ede8b))
('p': (a9c6b0b)-[:INTERACTS {weight:8}]->(f4d2ceb)-[:INTERACTS {weight:5}]->(cb2f184)<-[:INTERACTS {weight:5}]-(b500c3e)-[:INTERACTS {weight:18}]->(c0ede8b))
('p': (a9c6b0b)-[:INTERACTS {weight:19}]->(eb5c770)-[:INTERACTS {weight:4}]->(c1b11c5)<-[:INTERACTS {weight:20}]-(b500c3e)-[:INTERACTS {weight:18}]->(c0ede8b))
('p': (a9c6b0b)-[:INTERACTS {weight:19}]->(eb5c770)-[:INTERACTS {weight:4}]->(c1b11c5)<-[:INTERACTS {weight:11}]-(f14a35f)-[:INTERACTS {weight:6}]->(c0ede8b))
('p': (a9c6b0b)-[:INTERACTS {weight:19}]->(eb5c770)-[:INTERACTS {weight:17}]->(cb2f184)<-[:INTERACTS {weight:5}]-(b500c3e)-[:INTERACTS {weight:18}]->(c0ede8b))
('p': (a9c6b0b)-[:INTERACTS {weight:5}]->(e733

![](http://www.lyonwj.com/public/img/allshortestpaths-got.png)

### Pivotal nodes

In [12]:
for r in graph.run('''
// Find all pivotal nodes in network
MATCH (a:Character), (b:Character)
MATCH p=allShortestPaths((a)-[:INTERACTS*]-(b)) WITH collect(p) AS paths, a, b
MATCH (c:Character) WHERE all(x IN paths WHERE c IN nodes(x)) AND NOT c IN [a,b]
RETURN a.name, b.name, c.name AS PivotalNode SKIP 490 LIMIT 10
'''):
    print(r)

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



DatabaseError: The shortest path algorithm does not work when the start and end nodes are the same. This can happen if you
perform a shortestPath search after a cartesian product that might have the same start and end nodes for some
of the rows passed to shortestPath. If you would rather not experience this exception, and can accept the
possibility of missing results for those rows, disable this in the Neo4j configuration by setting
`cypher.forbid_shortestpath_common_node` to false. If you cannot accept missing results, and really want the
shortestPath between two common nodes, then re-write the query using a standard Cypher variable length pattern
expression followed by ordering by path length and limiting to one result.

In [13]:
for r in graph.run('''
MATCH (a:Character {name: "Drogo"}), (b:Character {name: "Ramsay"})
MATCH p=allShortestPaths((a)-[:INTERACTS*]-(b))
RETURN p
'''):
    print(r)

('p': (c0ede8b)<-[:INTERACTS {weight:18}]-(b500c3e)-[:INTERACTS {weight:5}]->(cb2f184)-[:INTERACTS {weight:5}]->(f2d24e3)<-[:INTERACTS {weight:4}]-(fbc71ea)-[:INTERACTS {weight:4}]->(d9491b7))
('p': (c0ede8b)<-[:INTERACTS {weight:18}]-(b500c3e)-[:INTERACTS {weight:5}]->(cb2f184)<-[:INTERACTS {weight:5}]-(f4d2ceb)<-[:INTERACTS {weight:15}]-(fbc71ea)-[:INTERACTS {weight:4}]->(d9491b7))
('p': (c0ede8b)<-[:INTERACTS {weight:18}]-(b500c3e)-[:INTERACTS {weight:20}]->(c1b11c5)<-[:INTERACTS {weight:4}]-(eb5c770)<-[:INTERACTS {weight:15}]-(fbc71ea)-[:INTERACTS {weight:4}]->(d9491b7))
('p': (c0ede8b)<-[:INTERACTS {weight:6}]-(f14a35f)-[:INTERACTS {weight:11}]->(c1b11c5)<-[:INTERACTS {weight:4}]-(eb5c770)<-[:INTERACTS {weight:15}]-(fbc71ea)-[:INTERACTS {weight:4}]->(d9491b7))
('p': (c0ede8b)<-[:INTERACTS {weight:18}]-(b500c3e)-[:INTERACTS {weight:5}]->(cb2f184)<-[:INTERACTS {weight:17}]-(eb5c770)<-[:INTERACTS {weight:15}]-(fbc71ea)-[:INTERACTS {weight:4}]->(d9491b7))
('p': (c0ede8b)<-[:INTERACTS 

![](http://www.lyonwj.com/public/img/pivotal-path.png)

## Centrality measures

### Degree centrality

In [14]:
for r in graph.run('''
MATCH (c:Character)-[:INTERACTS]-()
RETURN c.name AS character, count(*) AS degree ORDER BY degree DESC LIMIT 10
'''):
    print(r)

('character': 'Tyrion', 'degree': 36)
('character': 'Jon', 'degree': 26)
('character': 'Sansa', 'degree': 26)
('character': 'Robb', 'degree': 25)
('character': 'Jaime', 'degree': 24)
('character': 'Tywin', 'degree': 22)
('character': 'Cersei', 'degree': 20)
('character': 'Arya', 'degree': 19)
('character': 'Joffrey', 'degree': 18)
('character': 'Robert', 'degree': 18)


### Weighted degree centrality

In [15]:
for r in graph.run('''
MATCH (c:Character)-[r:INTERACTS]-()
RETURN c.name AS character, sum(r.weight) AS weightedDegree ORDER BY weightedDegree DESC LIMIT 10
'''):
    print(r)

('character': 'Tyrion', 'weightedDegree': 551)
('character': 'Jon', 'weightedDegree': 442)
('character': 'Sansa', 'weightedDegree': 383)
('character': 'Jaime', 'weightedDegree': 372)
('character': 'Bran', 'weightedDegree': 344)
('character': 'Robb', 'weightedDegree': 342)
('character': 'Samwell', 'weightedDegree': 282)
('character': 'Arya', 'weightedDegree': 269)
('character': 'Joffrey', 'weightedDegree': 255)
('character': 'Daenerys', 'weightedDegree': 232)


### Betweenness centrality

In [16]:
for r in graph.run('''
MATCH (c:Character)
WITH collect(c) AS characters
CALL apoc.algo.betweenness(['INTERACTS'], characters, 'BOTH') YIELD node, score
SET node.betweenness = score
RETURN node.name AS name, score ORDER BY score DESC LIMIT 10
'''):
    print(r)

('name': 'Jon', 'score': 1279.7533534055322)
('name': 'Robert', 'score': 1165.6025171231622)
('name': 'Tyrion', 'score': 1101.384972423435)
('name': 'Daenerys', 'score': 874.8372110508583)
('name': 'Robb', 'score': 706.5572832464792)
('name': 'Sansa', 'score': 705.1985623519138)
('name': 'Stannis', 'score': 571.5247305125713)
('name': 'Jaime', 'score': 556.1852522889822)
('name': 'Arya', 'score': 443.01358430043337)
('name': 'Tywin', 'score': 364.7212195528085)


### Closeness centrality

In [17]:
for r in graph.run('''
MATCH (c:Character)
WITH collect(c) AS characters
CALL apoc.algo.closeness(['INTERACTS'], characters, 'BOTH') YIELD node, score
RETURN node.name AS name, score ORDER BY score DESC LIMIT 10
'''):
    print(r)

('name': 'Tyrion', 'score': 0.004830917874396135)
('name': 'Sansa', 'score': 0.004807692307692308)
('name': 'Robert', 'score': 0.0047169811320754715)
('name': 'Robb', 'score': 0.004608294930875576)
('name': 'Arya', 'score': 0.0045871559633027525)
('name': 'Jaime', 'score': 0.004524886877828055)
('name': 'Stannis', 'score': 0.004524886877828055)
('name': 'Jon', 'score': 0.004524886877828055)
('name': 'Tywin', 'score': 0.004424778761061947)
('name': 'Eddard', 'score': 0.004347826086956522)


## Using python-igraph

### Building an igraph instance from Neo4j

In [18]:
#! pip install python-igraph
from igraph import Graph as IGraph

query = '''
MATCH (c1:Character)-[r:INTERACTS]->(c2:Character)
RETURN c1.name, c2.name, r.weight AS weight
'''

ig = IGraph.TupleList(graph.run(query), weights=True)

ig

### PageRank

![](http://www.lyonwj.com/public/img/page-rank.png)



In [19]:
pg = ig.pagerank()
pgvs = []
for p in zip(ig.vs, pg):
    pgvs.append({"name": p[0]["name"], "pg": p[1]})
print(pgvs[:5])

write_clusters_query = '''
UNWIND {nodes} AS n
MATCH (c:Character) WHERE c.name = n.name
SET c.pagerank = n.pg
'''

graph.run(write_clusters_query, nodes=pgvs)


[{'name': 'Stannis', 'pg': 0.018020131765195593}, {'name': 'Aemon', 'pg': 0.007328980991947571}, {'name': 'Robert', 'pg': 0.022292016521362857}, {'name': 'Jon', 'pg': 0.035828696691635555}, {'name': 'Alliser', 'pg': 0.005162125869510499}]


In [20]:
for r in graph.run('''
MATCH (n:Character)
RETURN n.name AS name, n.pagerank AS pagerank ORDER BY pagerank DESC LIMIT 10
'''):
    print(r)

('name': 'Tyrion', 'pagerank': 0.04288498199996332)
('name': 'Jon', 'pagerank': 0.035828696691635555)
('name': 'Robb', 'pagerank': 0.030171146655947636)
('name': 'Sansa', 'pagerank': 0.030009716660108574)
('name': 'Daenerys', 'pagerank': 0.028814254258302738)
('name': 'Jaime', 'pagerank': 0.028727587587471203)
('name': 'Tywin', 'pagerank': 0.025700162626425407)
('name': 'Robert', 'pagerank': 0.022292016521362857)
('name': 'Cersei', 'pagerank': 0.022287327589773486)
('name': 'Arya', 'pagerank': 0.02205020966384447)


### Community detection

![](http://www.lyonwj.com/public/img/community-1.png)

In [21]:
clusters = IGraph.community_walktrap(ig, weights="weight").as_clustering()

nodes = [{"name": node["name"]} for node in ig.vs]
for node in nodes:
    idx = ig.vs.find(name=node["name"]).index
    node["community"] = clusters.membership[idx]

print(nodes[:5])

write_clusters_query = '''
UNWIND {nodes} AS n
MATCH (c:Character) WHERE c.name = n.name
SET c.community = toInt(n.community)
'''

graph.run(write_clusters_query, nodes=nodes)

[{'name': 'Stannis', 'community': 0}, {'name': 'Aemon', 'community': 1}, {'name': 'Robert', 'community': 2}, {'name': 'Jon', 'community': 1}, {'name': 'Alliser', 'community': 1}]


In [22]:
for r in graph.run('''
MATCH (c:Character)
WITH c.community AS cluster, collect(c.name) AS  members
RETURN cluster, members ORDER BY cluster ASC
'''):
    print(r)

('cluster': 0, 'members': ['Davos', 'Melisandre', 'Shireen', 'Stannis', 'Cressen', 'Salladhor'])
('cluster': 1, 'members': ['Aemon', 'Alliser', 'Craster', 'Eddison', 'Gilly', 'Janos', 'Jon', 'Mance', 'Rattleshirt', 'Samwell', 'Val', 'Ygritte', 'Grenn', 'Karl', 'Bowen', 'Dalla', 'Orell', 'Qhorin', 'Styr'])
('cluster': 2, 'members': ['Aerys', 'Amory', 'Balon', 'Brienne', 'Bronn', 'Cersei', 'Gregor', 'Jaime', 'Joffrey', 'Jon Arryn', 'Kevan', 'Loras', 'Lysa', 'Meryn', 'Myrcella', 'Oberyn', 'Podrick', 'Renly', 'Robert', 'Robert Arryn', 'Sansa', 'Shae', 'Tommen', 'Tyrion', 'Tywin', 'Varys', 'Walton', 'Petyr', 'Elia', 'Ilyn', 'Pycelle', 'Qyburn', 'Margaery', 'Olenna', 'Marillion', 'Ellaria', 'Mace', 'Chataya', 'Doran'])
('cluster': 3, 'members': ['Arya', 'Beric', 'Eddard', 'Gendry', 'Sandor', 'Anguy', 'Thoros'])
('cluster': 4, 'members': ['Brynden', 'Catelyn', 'Edmure', 'Hoster', 'Lothar', 'Rickard', 'Robb', 'Roose', 'Walder', 'Jeyne', 'Roslin', 'Ramsay'])
('cluster': 5, 'members': ['Belwas',

## Visualization

![](http://www.lyonwj.com/public/img/graph-of-thrones.png)

See [neovis.js](https://github.com/johnymontana/neovis.js)